<a href="https://colab.research.google.com/github/rfaylona/big-data-challenge/blob/main/amzon_pc_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-12 00:30:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.17MB/s    in 0.2s    

2022-04-12 00:30:29 (5.17 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amzn_PC_Reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
pc_reviews = spark.read.csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

pc_reviews.show(10)
pc_reviews.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

6908554

In [5]:
# checking and removing null values and load in col dependencies

from pyspark.sql.functions import col,isnan, when, count, to_date
pc_reviews.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in pc_reviews.columns]
   ).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               3|          3|            3|          3|   3|                3|              9|        383|        214|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [6]:
# check dataset size

drp_pc_rev = pc_reviews.dropna()
print((drp_pc_rev.count(), len(drp_pc_rev.columns)))

(6908145, 15)


In [8]:
# match and clean dataframe to schema

drp_pc_rev.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
# review table

review_id_table = drp_pc_rev.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3ARRMDEGED8RD|   22873041|B00KJWQIIC|     335625766| 2015-08-31|
| RQ28TSA020Y6J|   30088427|B013ALA9LA|     671157305| 2015-08-31|
| RUXJRZCT6953M|   20329786|B00PML2GQ8|     982036237| 2015-08-31|
| R7EO0UO6BPB71|   14215710|B001NS0OZ4|     576587596| 2015-08-31|
|R39NJY2YJ1JFSV|   38264512|B00AQMTND2|     964759214| 2015-08-31|
|R31SR7REWNX7CF|   30548466|B00KX4TORI|     170101802| 2015-08-31|
| RVBP8I1R0CTZ8|     589298|B00P17WEMY|     206124740| 2015-08-31|
|R1QF6RS1PDLU18|   49329488|B00TR05L9Y|     778403103| 2015-08-31|
|R23AICGEDAJQL1|   50728290|B0098Y77OG|     177098042| 2015-08-31|
|R2EY3N4K9W19UP|   37802374|B00IFYEYXC|     602496520| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [10]:
# product table

products = drp_pc_rev.select(["product_id", "product_title"]).drop_duplicates()
products.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0016P392O|iMicro SP-IM942 M...|
|B00PPRYEZA|CoulVue Holder an...|
|B00VZXRRGG|Hexagon Network -...|
|B007GF9N96|Computer USB-Powe...|
|B00LCK3TEG|           Fire HD 6|
|B00KMOG5FS|Fintie 10.1" Tabl...|
|B00CFL5CRI|Cable Matters 2-P...|
|B008NP2AUC|USB USB-2, USB-3 ...|
|B00OAJ5N6I|Samsung 850 EVO -...|
|B00H02XN2I|Techno Earth Rubb...|
+----------+--------------------+
only showing top 10 rows



In [11]:
# customer table

customers = drp_pc_rev.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|            11|
|   14991350|             6|
|    9002099|             1|
|   35535911|             3|
|    1117644|             1|
|   46909180|             2|
|   48670265|             2|
|   43626894|             2|
|   17083191|             1|
|   20808998|             1|
+-----------+--------------+
only showing top 10 rows



In [13]:
# vine table

vine_table = drp_pc_rev.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3ARRMDEGED8RD|          5|            0|          0|   N|
| RQ28TSA020Y6J|          5|           24|         31|   N|
| RUXJRZCT6953M|          1|            2|          2|   N|
| R7EO0UO6BPB71|          1|            0|          0|   N|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|
|R31SR7REWNX7CF|          5|            0|          0|   N|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|
|R1QF6RS1PDLU18|          4|            1|          1|   N|
|R23AICGEDAJQL1|          1|            0|          0|   N|
|R2EY3N4K9W19UP|          5|            3|          4|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



**PostGres Setup**

In [14]:
# Configure settings for RDS

mode = "append"
jdbc_url="jdbc:postgresql://mypostgresschallengedb.cj9ms8auykiz.us-west-2.rds.amazonaws.com:5432/data_level_one"
config = {"user":"root", 
          "password": "simplepass", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write reviews to table in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)